In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import random as rd

Для начала напишем функцию поиска 1 доминирующего элемента в потоке (встречается > n / 2 раз). В случае, если условие не выполняется, функция может вернуть любой элемент

In [3]:
def dominant_simple(gen):
    iter_gen = iter(gen)
    dominant = next(iter_gen)
    dominant_count = 1
    for item in iter_gen:
        if dominant_count == 0:
            dominant = item
            dominant_count = 1
        elif dominant == item:
            dominant_count += 1
        else:
            dominant_count -= 1
    return dominant

Протестируем:

In [4]:
dominant_simple([10, 1, 10, 1, 2, 3, 4, 1, 1 ,1, 5, 1, 1, 1, 5])

1

In [12]:
dominant_simple([10, 2, 10, 5, 2, 3, 4, 1, 1 ,1, 5, 10, 1, 5, 10]) # тут можем получить и неверный ответ

10

Теперь тест в большем масштабе - определим генератор с гарантированно преобладающим элементом

In [6]:
def gen_gen(N):
    i = 0
    for v in range(N):
        yield rd.randint(1, 10) if i % 2 == 0 else 1
        i += 1
    yield 1

In [7]:
dominant_simple(gen_gen(2**20))

1

Теперь реализуем алгоритм поиска элементов, встречающихся более n / k раз

In [92]:
def dominant_k(gen, k):
    dominants = {}
    iter_gen = iter(gen)
    for _ in iter_gen:
        if _ in dominants:
            dominants[_] += 1
        elif len(dominants) < k - 1:
            dominants[_] = 1
        else:
            lst = list(dominants.keys())
            for key in lst:
                dominants[key] -= 1
                if dominants[key] == 0:
                    del dominants[key]
    return dominants.keys()

Тестируем:

In [100]:
dominant_k([3, 1, 3, 2, 2, 1, 2, 3, 3, 2], 3)

dict_keys([2, 3])

Алгоритм возвращает k - 1 значение, встречавшееся в потоке > n / k раз. Если таких значений было < n / k - вернёт что угодно (но в списке всегда присутсвуют значения встретившиеся > n / k раз)

In [102]:
dominant_k([1, 2, 1, 2, 1, 3, 2, 1, 3, 1], 2) # пример, в котором нет доминируещего элемента

dict_keys([])

In [113]:
def gen_gen1(N): # поток, как минимум на 1/3 состоящий из 1, и на 1/3 из 2
    i = 0
    for v in range(N):
        if i % 3 == 0:
            yield 1
        elif i % 3 == 1:
            yield 2
        else:
            yield rd.randint(1, 10)
        i += 1

In [112]:
dominant_k(gen_gen1(2**20), 3)

dict_keys([1, 2])

Получили корректный результат